In [ ]:
import os
import csv
import time
import math
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
import requests
from bs4 import BeautifulSoup
import re
import random
'''
使用进程池爬取解析及存储解析结果示例
爬取链家网天津二手房成交数据，并保存在'data/lianjia'目录下
'''


class CrawlProcess(object):
    '''
    配合进程池进行URL链接爬取及结果解析；
    最终通过crawl方法的complete_callback参数进行爬取解析结果回调
    '''
    def __init__(self):  
        self.proxies = {"http":"http://221.14.140.130:80"}
        self.headers = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36"
                        " (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"}
        self.iplist = [{'https':'https://42.51.12.2:808'},
                         {'https':'https://182.202.221.101:61234'},
                         {'http':'http://117.60.161.136:61234'},
                         {'http':'http://182.202.223.41:61234'},
                         {'https':'https://1.196.161.170:9999'},
                         {'https':'https://124.237.83.14:53281'},
                         {'http':'http://125.46.0.62:53281'},
                         {'https':'https://120.77.254.116:3128'},
                         {'http':'http://218.241.234.48:8080'},
                         {'http':'http://203.174.112.13:3128'},
                         {'https':'https://222.222.169.60:53281'},
                         {'https':'https://202.38.92.100:3128'},
                         {'https':'https://118.212.137.135:31288'},
                         {'https':'https://139.224.80.139:3128'},
                         {'https':'https://122.72.18.35:80'},
                         {'https':'https://119.28.152.208:80'},
                         {'http':'http://113.200.214.164:9999'},
                         {'https':'https://123.161.16.20:9797'},
                         {'https':'https://114.215.95.188:3128'},
                         {'https':'https://122.72.18.34:80'},
                         {'https':'https://125.66.165.155:53281'},
                         {'https':'https://122.136.212.132:53281'},
                         {'http':'http://113.106.195.98:8088'},
                         {'https':'https://101.37.79.125:3128'}]
        #伪装浏览器头
        self.user_agent_list = ["Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36",
                                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
                                "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
                                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
                                "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
                                "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
                                "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
                                "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
                                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
                                "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
                                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
                                "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
                                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
                                "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
                                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
                                "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
                                "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
                                "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
                                "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
                                ]

    def get(self,url, timeout =5, proxy=None, num_retries=10):
        UA = random.choice(self.user_agent_list)  ##从user_agent_list中随机取出一个字符串
        headers = {'User-Agent':UA}
        if proxy == None:
            try:
                return requests.get(url, headers=self.headers,timeout=timeout)  ##这样服务器就会以为我们是真的浏览器了
            except:  ##如过上面的代码执行报错则执行下面的代码
                if num_retries > 0:  ##num_retries是我们限定的重试次数
                    time.sleep(3)  ##延迟3秒
                    print('获取网页出错，3S后将获取倒数第：', num_retries, '次')
                    return get(url, timeout,None,num_retries - 1)  ##调用自身 并将次数减1
                else:
                    print('开始使用代理')
                    time.sleep(3)
                    proxy = random.choice(self.iplist)
                    return get(url,proxy=proxy)  ##代理不为空的时候
        else:
            try:
                proxy = random.choice(self.iplist) ##将从self.iplist中获取的字符串处理成我们需要的格式（处理了些什么自己看哦，这是基础呢）
                return requests.get(url, headers=self.headers, proxies=proxy,timeout=timeout)  ##使用代理获取response
            except:
                if num_retries > 0:
                    time.sleep(3)
                    proxy = random.choice(self.iplist)
                    print('正在更换代理，3S后将重新获取倒数第', num_retries, '次')
                    print('当前代理是：', proxy)
                    return get(url, timeout, proxy, num_retries - 1)
                else:
                    print('代理也不好使了！取消代理')
                    return get(url)

    def _request_parse_runnable(self, turl):
        try:
            result = []
            retu = {}
            isInRoad = {'东丽': ['二号桥街', '月牙河','钢管公司'],
                         '北辰': ['双环邨', '武清其他', '杨柳青'],
                         '南开': ['大胡同', '鸿顺里街', '南市', '三条石', '铁东路', '天穆镇', '新开河', '小淀镇', '宜兴埠'],
                         '塘沽': ['滨海其他','三槐路街','新城镇'],
                         '河东': ['靖江路', '万新街', '张贵庄','鲁山道'],
                         '河北': ['靖江路', '真理道'],
                         '津南': ['大寺', '尖山', '柳林街', '梅江', '新梅江', '小海地','葛沽镇'],
                         '西青': ['八里台', '华苑', '梅江', '双口镇', '双港', '体育中心街', '王顶堤', '新梅江', '向阳路','辛口镇','西堤头'],
                         '武清': ['静湖']}
            url = turl.split('-')[0]
            area = turl.split('-')[1]
            road = turl.split('-')[2]
            isroad = turl.split('-')[3]
            count_area = int(turl.split('-')[4])
            count_road = int(turl.split('-')[5])
            if isroad == "True":
                rep_road = requests.get(url=url,headers=self.headers,proxies=self.proxies)
#                 rep_road = self.get(url=url,proxy=1)
                sp_road = BeautifulSoup(rep_road.text,'lxml')
                post_area = sp_road.select('.position')[0].select('div')[0].select('a')[count_area]
                area = post_area.text    
                post_road = sp_road.select('.position')[0].select('div')[0].select('div')[1].select('a')[count_road]
                url = 'https://tj.lianjia.com'+post_road['href']
                print('A:',url)
                road = post_road.text
                isroad = 'False'
#                 count_road += 1
            time.sleep(1)
            response = requests.get(url=url,headers=self.headers,proxies=self.proxies)
#             response = self.get(url=url,proxy=1)
            soup = BeautifulSoup(response.text, "lxml")
            for info in soup.select('.img'):
                result.append(info['href'])
            text = str(soup.select('.page-box')[0])
            totalPage = int(re.findall(re.compile(r"totalPage\":.*?(\d+)"),text)[0])
            curPage = int(re.findall(re.compile(r"curPage\":.*?(\d+)"),text)[0])
            href = url.split('/')
            href[5] = '%s%d%s'%('pg',(curPage+1),'ddo21')
            url = "/".join(href)
            if totalPage == curPage:
                count_road += 1
                while True:
                    if len(soup.select('.position')[0].select('div')[0].select('div')[1].select('a')) > count_road:
                        if soup.select('.position')[0].select('div')[0].select('a')[count_area].text in list(isInRoad.keys()):
                            if soup.select('.position')[0].select('div')[0].select('div')[1].select('a')[count_road].text in isInRoad.get(area):
                                count_road += 1
                            else:
                                break
                        else:
                            break
                    else:
                        break
                if len(soup.select('.position')[0].select('div')[0].select('div')[1].select('a')) > count_road:
                    post_road = soup.select('.position')[0].select('div')[0].select('div')[1].select('a')[count_road]
                    url = 'https://tj.lianjia.com'+post_road['href']
                    print('B:',url)
                    isroad = 'True'
                else:
                    count_area += 1
                    post_area = soup.select('.position')[0].select('div')[0].select('a')[count_area]
                    url = 'https://tj.lianjia.com'+post_area['href']
                    print('C:',url)
                    isroad = 'True'
                    count_road = 0
            retu['url'] = '%s-%s-%s-%s-%d-%d'%(url,area,road,isroad,count_area,count_road)
            retu['urls'] = result
        except BaseException as e:
            print('connection error.'+str(e))
            print(url)
            retu = None
        return retu

    def crawl(self, url, complete_callback, process_pool):
        future = process_pool.submit(self._request_parse_runnable, url)
        future.add_done_callback(complete_callback)


class OutPutProcess(object):
    '''
    配合进程池对上面爬取解析进程结果进行进程池处理存储；
    '''
    def _output_runnable(self, crawl_result):
        try:
            if crawl_result == None:
                return crawl_result
            save_dir = 'data/lianjia'
            if os.path.exists(save_dir) is False:
                os.makedirs(save_dir)
            save_file = save_dir + os.path.sep + 'LJdata' + '.csv'
            if os.path.exists(save_file):
                csv = pd.DataFrame.from_dict(crawl_result,orient='index').T
                csv.to_csv(save_file,index=False,header=False,mode='a+',encoding="gbk")
            else:
                csv = pd.DataFrame.from_dict(crawl_result,orient='index').T
                csv.to_csv(save_file,index=False,header=True,encoding='gbk')
        except Exception as e:
            print('save file error.'+str(e))
        return crawl_result

    def save(self, crawl_result, process_pool):
        process_pool.submit(self._output_runnable, crawl_result)

class ParserProcess(object):
    '''
    获取内容类，获取房子详细信息
    '''
    def __init__(self):  
        self.proxies = {"http":"http://221.14.140.130:80"}
        self.headers = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36"
                        " (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"}
    
    def _content_runnable(self,url):
        try:
            content_result = {}
            time.sleep(1)
            area = url.split('-')[0]
            road = url.split('-')[1]
            url = url.split('-')[2]
            response = requests.get(url=url,headers=self.headers,proxies=self.proxies)
#             response = self.get(url=url,proxy=1)
            soup = BeautifulSoup(response.text, "lxml")
            content_result['地区'] = area
            content_result['街道'] = road
            content_result['小区名'] = soup.select('.index_h1')[0].text.split(' ')[0]
            for roominfo in soup.select('.content'):
                for li in roominfo.select('li'):
                    if li.select('.label')[0].text == '建筑面积':
                        content_result[li.select('.label')[0].text+'(平方米)']=li.text.strip(li.select('.label')[0].text).strip().strip('㎡')
                    elif li.select('.label')[0].text == '套内面积':
                        if li.text.strip(li.select('.label')[0].text).strip() == '暂无数据':
                            content_result[li.select('.label')[0].text+'(平方米)']=li.text.strip(li.select('.label')[0].text).strip()
                        else:
                            content_result[li.select('.label')[0].text+'(平方米)']=li.text.strip(li.select('.label')[0].text).strip().strip('㎡')
                    elif li.select('.label')[0].text == '链家编号':
                        pass
                    elif li.select('.label')[0].text == '产权年限':
                        content_result[li.select('.label')[0].text+"(年)"]=li.text.strip(li.select('.label')[0].text).strip().strip('年')
                    elif li.select('.label')[0].text == '所在楼层':
                        content_result[li.select('.label')[0].text] = li.text.strip(li.select('.label')[0].text).strip().split('(')[0].strip()
                        content_result['共几层楼'] = int(re.findall(re.compile(r"\d+"),li.text.strip(li.select('.label')[0].text).strip())[0])
                    else:
                        content_result[li.select('.label')[0].text]=li.text.strip(li.select('.label')[0].text).strip()
            content_result['总价(万)'] = soup.select('.record_price')[0].text.strip('万')
            content_result['单价(元/平方米)'] = soup.select('.record_detail')[0].text.split(',')[0].strip('单价').strip('元/平')
            content_result['成交日期'] = soup.select('.record_detail')[0].text.split(',')[2]
            content_result['挂牌价格(万)'] = soup.select('.msg')[0].select('label')[0].text
            content_result['成交周期(天)'] = soup.select('.msg')[0].select('label')[1].text
            content_result['调价次数(次)'] = soup.select('.msg')[0].select('label')[2].text
            content_result['关注人数(人)'] = soup.select('.msg')[0].select('label')[4].text
            content_result['浏览次数(次)'] = soup.select('.msg')[0].select('label')[5].text
        except Exception as e:
            print('parser error.'+str(e))
            print(url)
            dict_url = {}
            dict_url['地区'] = area
            dict_url['街道'] = road
            dict_url['ErrorUrl'] = url
            try:
                save_dir = 'data/lianjia'
                if os.path.exists(save_dir) is False:
                    os.makedirs(save_dir)
                save_file = save_dir + os.path.sep + 'ErrorUrls' + '.csv'
                if os.path.exists(save_file):
                    csv = pd.DataFrame.from_dict(dict_url,orient='index').T
                    csv.to_csv(save_file,index=False,header=False,mode='a+',encoding="gbk")
                else:
                    csv = pd.DataFrame.from_dict(dict_url,orient='index').T
                    csv.to_csv(save_file,index=False,header=True,encoding='gbk')
            except Exception as e:
                print('save error url error.'+str(e))
        return content_result
    
    def content(self,url, complete_callback, process_pool):
        future = process_pool.submit(self._content_runnable, url)
        future.add_done_callback(complete_callback)
    
class CrawlManager(object):
    '''
    爬虫管理类，进程池负责统一管理调度爬取解析及存储进程
    '''
    def __init__(self):
        self.crawl = CrawlProcess()
        self.parser = ParserProcess()
        self.output = OutPutProcess()
        self.crawl_pool = ProcessPoolExecutor(max_workers=8)
        self.crawl_deep = 2322   #爬取深度
        self.crawl_cur_count = 0

    def _crawl_future_callback(self, crawl_url_future):
        try:
            res = crawl_url_future.result()
            urls = res.get('urls')
            url = res.get('url')
            area = url.split('-')[1]
            road = url.split('-')[2]
            for href in urls:
                href = '%s-%s-%s'%(area,road,href)
                self.parser.content(href,self._parser_callback,self.crawl_pool)
            self.start_runner(url)
        except Exception as e:
            print('Run crawl url future process error. '+str(e))
            print(url)
    
    def _parser_callback(self,parser_content_future):
        try:
            data = parser_content_future.result()
            self.output.save(data,self.crawl_pool)
        except Exception as e:
            print('Run parser contetn future process error.'+str(e))

    def start_runner(self, url):
        if self.crawl_cur_count > self.crawl_deep:
            print('结束！')
            return
        print(self.crawl_cur_count+1)
        self.crawl_cur_count += 1
        self.crawl.crawl(url, self._crawl_future_callback, self.crawl_pool)

if __name__ == '__main__':
    root_url = '%s-%s-%s-%s-%d-%d'%('https://tj.lianjia.com/chengjiao/heping/ddo21/','','','True',0,0)
    CrawlManager().start_runner(root_url)